# Multihead Attention; matrix profile forward


https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb

Lets test multiple attention heads, one for each input, with univariate and mutivariate data

Due to a slip in the matrix profile function, the matrix profile was computed in reverse in the previous iteration, this is adressed and computed normally in this notebook

## Load data, functions

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import MultiHeadAttention, BatchNormalization, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pywt
import matrixprofile as mp
from keras import Model
from keras.layers import dot
import datetime 
from datetime import timedelta

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [8]:
def generate_training_val_examples(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    #we are only lookign to forecast the min gas price
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))

    
    return X_train, y_train, X_val, y_val, scaler



In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [10]:
def descale_y_retrun_metrics(yhat, y_val2):
    #We will use validation data that has not had outleirs limited, will be a different min/max scaler as such
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler2.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

In [11]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler

In [12]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    yhat_stretched= np.repeat(yhat[:,:,0],len(inputs)).reshape(yhat.shape[0], yhat.shape[1], len(inputs))
    y_val2_stretched= np.repeat(y_val2[:,:,0],len(inputs)).reshape(y_val2.shape[0], y_val2.shape[1], len(inputs))
    
    for j in range(0, n_steps_out):
        
        
        pred_descaled= (scaler.inverse_transform(yhat_stretched[:, j:j+1, :].reshape(yhat_stretched.shape[0], yhat_stretched.shape[2])))[:,:1]
        groud_truth_descaled= (scaler.inverse_transform(array([y_val2[ :, j:j+1,0].reshape(y_val2.shape[0])]*len(inputs)).transpose()))[:,:1]
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [13]:
def add_mp(data, window):
    #Given 3d array, add matrix profile of (x,y,0) as new dimension; new array has dimensiosn (x,y,z+1) 
    mp_list=[]
    for i in data[:,:,0]:
        profile = mp.compute(i, window, n_jobs=4)['mp']
        #we are padding the end of the sequence with the mean
        #matrix profile is always 1 full window size smalelr than input data
        mp_list.append(np.append(([mean(profile)]*(data.shape[1]-len(profile))),profile))
        
    #concatenate matrix profile data with original    
    mp_array = np.array(mp_list).reshape(data.shape[0], data.shape[1])
    std_array = ((mp_array-mean(mp_array))/np.std(mp_array)).reshape(data.shape[0], data.shape[1],1)
    data = np.concatenate((data, std_array), axis=2)[:, window:, :]
    
    return data

## Univariate + MP

In [ ]:
def LSTM_model(): 
    
    #set up callback for best val loss model
    checkpoint_filepath='./cnn/checkpoint'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    
    n_hidden = 30
    input_train = Input(shape=(n_steps_in, X_train.shape[2]),name='input')
    output_train = Input(shape=( y_train.shape[1], y_train.shape[2]))




    enc_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('encoder' +str(i)))(input_train)

        decoder_input = RepeatVector(y_train.shape[1], name='repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('attention_dot'+str(i)))
        attention = Activation('softmax', name='attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='Context'+str(i))
        enc_head_list.append(context)
    enc_concat_attention = Concatenate(axis=2)(enc_head_list)
    
    
    
    dec_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('decoder' +str(i)))(enc_concat_attention)

        decoder_input = RepeatVector(y_train.shape[1],name='dec_repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('dec_alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('dec_attention_dot'+str(i)))
        attention = Activation('softmax', name='dec_attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='dec_Context'+str(i))
        dec_head_list.append(context)
    dec_concat_attention = Concatenate(axis=2)(dec_head_list)





    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)




    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)

    model = Model(inputs=input_train, outputs=out)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])
    return model, model_checkpoint_callback, checkpoint_filepath

In [26]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 5
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window = 288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('Univariate_MP_forward/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('Univariate_MP_forward/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('Univariate_MP_forward/val_loss.csv')
    np.save("Univariate_MP_forward/training_metrics.npy", training_metrics_dicts)
    np.save("Univariate_MP_forward/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 49s 293ms/step - loss: 0.2609 - mae: 0.2474 - val_loss: 0.0769 - val_mae: 0.1873
Epoch 2/15
101/101 [==============================] - 25s 245ms/step - loss: 0.2264 - mae: 0.2096 - val_loss: 0.0926 - val_mae: 0.2311
Epoch 3/15
101/101 [==============================] - 25s 246ms/step - loss: 0.2211 - mae: 0.2051 - val_loss: 0.0793 - val_mae: 0.2084
Epoch 4/15
101/101 [==============================] - 25s 245ms/step - loss: 0.2198 - mae: 0.2040 - val_loss: 0.0851 - val_mae: 0.2196
Epoch 5/15
101/101 [==============================] - 25s 246ms/step - loss: 0.2167 - mae: 0.2018 - val_loss: 0.0778 - val_mae: 0.2148
Epoch 6/15
101/101 [==============================] - 25s 248ms/step - loss: 0.2134 - mae: 0.2022 - val_loss: 0.0748 - val_mae: 0.2042
Epoch 7/15
101/101 [==============================] - 111s 1s/step - loss: 0.2118 - mae: 0.2002 - val_loss: 0.0626 - val_mae: 0.1765
Epoch 8/15
101/101 [==============================] - 89s

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 46s 294ms/step - loss: 0.7622 - mae: 0.5178 - val_loss: 0.4542 - val_mae: 0.4545
Epoch 2/15
107/107 [==============================] - 27s 252ms/step - loss: 0.5710 - mae: 0.4037 - val_loss: 0.4166 - val_mae: 0.4227
Epoch 3/15
107/107 [==============================] - 27s 251ms/step - loss: 0.5485 - mae: 0.3836 - val_loss: 0.4151 - val_mae: 0.3541
Epoch 4/15
107/107 [==============================] - 27s 249ms/step - loss: 0.5217 - mae: 0.3738 - val_loss: 0.3847 - val_mae: 0.3776
Epoch 5/15
107/107 [==============================] - 27s 250ms/step - loss: 0.5173 - mae: 0.3680 - val_loss: 0.4053 - val_mae: 0.3710
Epoch 6/15
107/107 [==============================] - 27s 252ms/step - loss: 0.5096 - mae: 0.3688 - val_loss: 0.4239 - val_mae: 0.3586
Epoch 7/15
107/107 [==============================] - 26s 247ms/step - loss: 0.5038 - mae: 0.3623 - val_loss: 0.4252 - val_mae: 0.3967
Epoch 8/15
107/107 [==============================] - 2

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 48s 309ms/step - loss: 0.1808 - mae: 0.2513 - val_loss: 0.4152 - val_mae: 0.3111
Epoch 2/15
101/101 [==============================] - 25s 248ms/step - loss: 0.1472 - mae: 0.2108 - val_loss: 0.3888 - val_mae: 0.2798
Epoch 3/15
101/101 [==============================] - 26s 256ms/step - loss: 0.1418 - mae: 0.2013 - val_loss: 0.3684 - val_mae: 0.2600
Epoch 4/15
101/101 [==============================] - 29s 283ms/step - loss: 0.1418 - mae: 0.2026 - val_loss: 0.3719 - val_mae: 0.2582
Epoch 5/15
101/101 [==============================] - 26s 255ms/step - loss: 0.1396 - mae: 0.1995 - val_loss: 0.3708 - val_mae: 0.2612
Epoch 6/15
101/101 [==============================] - 25s 243ms/step - loss: 0.1374 - mae: 0.1977 - val_loss: 0.3840 - val_mae: 0.2703
Epoch 7/15
101/101 [==============================] - 25s 247ms/step - loss: 0.1375 - mae: 0.1991 - val_loss: 0.3778 - val_mae: 0.2716
Epoch 8/15
101/101 [==============================] - 2

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
82/82 [==============================] - 42s 283ms/step - loss: 0.2340 - mae: 0.2412 - val_loss: 0.0793 - val_mae: 0.1926
Epoch 2/15
82/82 [==============================] - 19s 236ms/step - loss: 0.1703 - mae: 0.1658 - val_loss: 0.0504 - val_mae: 0.1706
Epoch 3/15
82/82 [==============================] - 19s 235ms/step - loss: 0.1563 - mae: 0.1557 - val_loss: 0.0484 - val_mae: 0.1603
Epoch 4/15
82/82 [==============================] - 19s 234ms/step - loss: 0.1505 - mae: 0.1463 - val_loss: 0.0449 - val_mae: 0.1532
Epoch 5/15
82/82 [==============================] - 19s 234ms/step - loss: 0.1485 - mae: 0.1431 - val_loss: 0.0475 - val_mae: 0.1632
Epoch 6/15
82/82 [==============================] - 19s 235ms/step - loss: 0.1495 - mae: 0.1457 - val_loss: 0.0438 - val_mae: 0.1461
Epoch 7/15
82/82 [==============================] - 19s 234ms/step - loss: 0.1458 - mae: 0.1418 - val_loss: 0.0479 - val_mae: 0.1493
Epoch 8/15
82/82 [==============================] - 19s 237ms/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 45s 277ms/step - loss: 0.1344 - mae: 0.2070 - val_loss: 0.0423 - val_mae: 0.1281
Epoch 2/15
101/101 [==============================] - 25s 245ms/step - loss: 0.0722 - mae: 0.1486 - val_loss: 0.0390 - val_mae: 0.1199
Epoch 3/15
101/101 [==============================] - 25s 249ms/step - loss: 0.0687 - mae: 0.1481 - val_loss: 0.0390 - val_mae: 0.1199
Epoch 4/15
101/101 [==============================] - 25s 249ms/step - loss: 0.0627 - mae: 0.1406 - val_loss: 0.0393 - val_mae: 0.1230
Epoch 5/15
101/101 [==============================] - 25s 251ms/step - loss: 0.0578 - mae: 0.1353 - val_loss: 0.0398 - val_mae: 0.1206
Epoch 6/15
101/101 [==============================] - 25s 251ms/step - loss: 0.0566 - mae: 0.1374 - val_loss: 0.0408 - val_mae: 0.1398
Epoch 7/15
101/101 [==============================] - 25s 252ms/step - loss: 0.0549 - mae: 0.1337 - val_loss: 0.0403 - val_mae: 0.1263
Epoch 8/15
101/101 [==============================] - 2

In [ ]:
y_train.shape

(3222, 5, 1)

In [ ]:
X_train.shape

(3222, 4320, 2)

## 2 layer, all variables, MP

In [14]:
def LSTM_model(): 
    
    #set up callback for best val loss model
    checkpoint_filepath='./cnn/checkpoint'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    
    n_hidden = 30
    input_train = Input(shape=(n_steps_in, X_train.shape[2]),name='input')
    output_train = Input(shape=( y_train.shape[1], y_train.shape[2]))




    enc_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('encoder' +str(i)))(input_train)

        decoder_input = RepeatVector(y_train.shape[1], name='repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('attention_dot'+str(i)))
        attention = Activation('softmax', name='attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='Context'+str(i))
        enc_head_list.append(context)
    enc_concat_attention = Concatenate(axis=2)(enc_head_list)
    
    
    
    dec_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('decoder' +str(i)))(enc_concat_attention)

        decoder_input = RepeatVector(y_train.shape[1],name='dec_repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('dec_alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('dec_attention_dot'+str(i)))
        attention = Activation('softmax', name='dec_attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='dec_Context'+str(i))
        dec_head_list.append(context)
    dec_concat_attention = Concatenate(axis=2)(dec_head_list)





    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)




    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)

    model = Model(inputs=input_train, outputs=out)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])
    return model, model_checkpoint_callback, checkpoint_filepath

In [29]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 5
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train = add_mp(X_train, mp_window)
    X_val = add_mp(X_val, mp_window)
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_2/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_forward/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_forward/val_loss.csv')
    np.save("2_Att_layer_MP_forward/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_forward/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15


KeyboardInterrupt: 